<a href="https://colab.research.google.com/github/menouarazib/llm/blob/main/Fennec_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Hugging Face libraries
!pip install -q datasets accelerate bitsandbytes

# Install peft, transformers, and trl from Github
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/peft
!pip install -q git+https://github.com/huggingface/trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparin

In [2]:
import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'

# Install flash-attn
!pip -q install ninja packaging
!MAX_JOBS=4 pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.9 MB/s eta 0:00:00


In [3]:
# Login into our HF account using our token
from huggingface_hub import login
from google.colab import userdata

login(
  token=userdata.get('HF_TOKEN'), # Retrieve my HF_TOKEN stored in Google Colab Secrets
  add_to_git_credential=True
)

# The id of my HF Repo
hf_repo_id = "fennec-7b-alpha"

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Load Efficiely the openbmb/UltraFeedback

In [4]:
def filter_gpt4_responses(example):
    # Check if 'gpt-4' is in the models list for this completion
    return 'gpt-4' in example['models']

In [5]:
from datasets import load_dataset

# Load the dataset in a streaming fashion
dataset = load_dataset('openbmb/UltraFeedback', streaming=True)
dataset = dataset['train']

In [6]:
# Apply the filter within your processing loop
filtered_dataset = (ex for ex in dataset if filter_gpt4_responses(ex))
filtered_dataset_list = list(filtered_dataset)

In [7]:
import pandas as pd
from datasets import Dataset

# Convert the list to a DataFrame
df = pd.DataFrame(filtered_dataset_list)

# Load the DataFrame as a Dataset
dataset1 = Dataset.from_pandas(df)
print(dataset1)

Dataset({
    features: ['source', 'instruction', 'models', 'completions', 'correct_answers', 'incorrect_answers'],
    num_rows: 16518
})


Load HuggingFaceH4/ultrachat_200k

In [8]:
from datasets import load_dataset

dataset2 = load_dataset("HuggingFaceH4/ultrachat_200k")

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

In [9]:
dataset2 = dataset2['train_sft']
print(dataset2)

Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 207865
})


Load GSM8K dataset

In [10]:
from datasets import load_dataset

dataset3 = load_dataset("gsm8k", 'main')
dataset3 = dataset3['train']

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Format the dataset and combine them

In [11]:
def format_dataset_1(sample):
  models = sample['models']
  gpt4_index = models.index('gpt-4')
  completion = sample['completions'][gpt4_index]

  return {
    "messages": [
      {"role": "system", "content": completion["custom_system_prompt"]},
      {"role": "user", "content": sample["instruction"]},
      {"role": "assistant", "content": completion['response']}
    ]
  }

def format_dataset_2(sample):
  messages = sample["messages"]
  messages.append({
    "role": "system",
    "content": "Welcome! I'm an AI assistant here to help you with any questions, provide information, or assist with tasks. Whether it's answering questions, offering guidance, or solving problems, I'm here for you. Please feel free to communicate clearly and respectfully. How can I assist you today?"
    })
  return {"messages": messages}

def format_dataset_3(sample):
   return {
    "messages": [
      {
    "role": "system",
    "content": "Hello! I'm a mathematics-focused AI assistant, ready to guide you through a wide range of mathematical problems step-by-step. From algebra and calculus to geometry and beyond, I'm here to help you tackle even the most challenging questions. Do you have any mathematical problems that require multi-step reasoning or any specific topics you'd like to explore? Feel free to ask, and let's solve them together!"
      },
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample['answer']}
    ]
  }

dataset1 = dataset1.map(format_dataset_1, remove_columns=dataset1.features)
dataset2 = dataset2.map(format_dataset_2, remove_columns=dataset2.features)
dataset3 = dataset3.map(format_dataset_3, remove_columns=dataset3.features)

Map:   0%|          | 0/16518 [00:00<?, ? examples/s]

Map:   0%|          | 0/207865 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [12]:
from datasets import concatenate_datasets

# Concatenate the datasets
combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3])
combined_dataset = combined_dataset.shuffle(seed=42)

In [13]:
print(combined_dataset)

Dataset({
    features: ['messages'],
    num_rows: 231856
})


In [14]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, MistralForCausalLM

# Hugging Face Mistral-7B model ID
model_id = "mistralai/Mistral-7B-v0.1"

# BitsAndBytesConfig for 4-bit integers
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model and tokenizer
model = MistralForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # to prevent warnings

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [15]:
from trl import setup_chat_format
model, tokenizer = setup_chat_format(model, tokenizer)

In [16]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [17]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=hf_repo_id,                  # repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=10,         # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard,
    max_steps=1000
)

In [18]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=combined_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)


Map:   0%|          | 0/231856 [00:00<?, ? examples/s]

In [19]:
model.config.use_cache = False
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,1.178500
20,0.990700
30,0.950300
40,0.938000
50,0.982800
60,0.902800
70,0.968600
80,0.987700
90,0.942700
100,0.979500


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [20]:
# Free the memory
del model
del trainer
torch.cuda.empty_cache()

from google.colab import runtime
runtime.unassign()